In [ ]:
import numpy as np
import pandas as pd
import time
import tracemalloc
from itertools import combinations
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error, median_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from statsmodels.tsa.arima.model import ARIMA
from keras.models import Sequential
from keras.layers import LSTM, GRU, SimpleRNN, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import xgboost as xgb
from scipy import stats
import warnings

warnings.filterwarnings("ignore")


#Exploratory Data Analysis (EDA) – CO₂ Emissions

Raw Time Series Visualization

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load your data

file = '/co2_preprocessed.xlsx'
data = pd.read_excel(file)

# Define your features
feature_keys = [
    "co2", "Consumption", "total_ghg_excluding_lucf", "GDP", "Energy consumption",
    "Electricity Supply", "Coal", "Gas", "Oil", "Transport", "AUS Energy Growth-QLD",
    "Total generation", "Residential_Energy", "Commercial_Energy",
    "AUS Energy Growth-Rest of Australia", "Renewables", "land_use_change_co2",
    "Net exports", "Energy intensity", "Energy productivity ",
    "Renewables generation", "AUS Energy Growth-NT", "population"
]

# Optional: use same titles as feature names (or customise them if needed)
titles = feature_keys

# Set the year as datetime index
date_time_key = "year"
data[date_time_key] = pd.to_datetime(data[date_time_key], format='%Y')
data.set_index(date_time_key, inplace=True)

# Color palette for plots
colors = plt.cm.tab20.colors

# Define visualization function with 3 subplots per row
def show_raw_visualization(data):
    time_data = data.index
    n_features = len(feature_keys)
    ncols = 3
    nrows = (n_features // ncols) + (n_features % ncols > 0)

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(22, nrows * 4), dpi=100, facecolor="w", edgecolor="k")
    axes = axes.flatten()

    for i, key in enumerate(feature_keys):
        c = colors[i % len(colors)]
        if key in data.columns:
            t_data = data[key]
            t_data.index = time_data
            ax = t_data.plot(ax=axes[i], color=c, title=titles[i], rot=25)
            ax.legend([titles[i]])
        else:
            print(f"Column '{key}' not found in DataFrame. Skipping...")

    # Hide unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

# Call the function
show_raw_visualization(data)

Scatter Plots – CO₂ vs Features

In [ ]:

# Set 'year' as datetime index
data["year"] = pd.to_datetime(data["year"], format="%Y")
data.set_index("year", inplace=True)

# Define features (excluding co2 from the list of "others")
feature_keys = [
    "Consumption", "total_ghg_excluding_lucf", "GDP", "Energy consumption",
    "Electricity Supply", "Coal", "Gas", "Oil", "Transport", "AUS Energy Growth-QLD",
    "Total generation", "Residential_Energy", "Commercial_Energy",
    "AUS Energy Growth-Rest of Australia", "Renewables", "land_use_change_co2",
    "Net exports", "Energy intensity", "Energy productivity ",
    "Renewables generation", "AUS Energy Growth-NT", "population"
]

# Plot settings
ncols = 3
nrows = (len(feature_keys) // ncols) + (len(feature_keys) % ncols > 0)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(22, nrows * 6))
axes = axes.flatten()

# Plot co2 vs each feature (scatter only)
for i, feature in enumerate(feature_keys):
    if "co2" in data.columns and feature in data.columns:
        sns.scatterplot(data=data, x=feature, y="co2", ax=axes[i], color='royalblue')
        axes[i].set_title(f"CO₂ vs {feature}", fontsize=12)
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel("CO₂")
    else:
        axes[i].axis('off')
        axes[i].text(0.5, 0.5, f"Missing: co2 or {feature}", ha='center')

# Hide any extra subplots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


Histograms of Features

In [ ]:
# Plot histograms for all features
def plot_histograms(data):
    fig, axes = plt.subplots(nrows=nrows_hist, ncols=ncols, figsize=(22, nrows_hist * 5), dpi=100, facecolor="w", edgecolor="k")
    axes = axes.flatten()

    for i, key in enumerate(feature_keys):
        if key in data.columns:
            ax = axes[i]
            data[key].plot(kind='hist', bins=20, ax=ax, color="skyblue", edgecolor="black", alpha=0.7)
            ax.set_title(f'Histogram: {key}', fontsize=12)
            ax.set_xlabel(key)
            ax.set_ylabel('Frequency')
        else:
            print(f"Feature '{key}' not found in DataFrame. Skipping...")

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

# Plot joint distributions for all features
def plot_joint_distributions(data):
    fig, axes = plt.subplots(nrows=nrows_joint, ncols=ncols, figsize=(22, nrows_joint * 5), dpi=100, facecolor="w", edgecolor="k")
    axes = axes.flatten()

    for i, key in enumerate(feature_keys):
        if key in data.columns:
            ax = axes[i]
            sns.jointplot(x=key, y=key, data=data, kind="scatter", color="skyblue", height=5)
            plt.suptitle(f'Joint Distribution: {key}', fontsize=12, y=1.02)
        else:
            print(f"Feature '{key}' not found in DataFrame. Skipping...")

    plt.tight_layout()
    plt.show()

# Call the functions to plot histograms and joint distributions
plot_histograms(data)
plot_joint_distributions(data)

#CO₂ Forecasting: Data & Models

In this section, we:

- Import all necessary libraries for data processing, machine learning, and deep learning.
- Load and preprocess CO₂ data from 1982–2015, including 22 key features like energy, economic, population, and land-use factors.
- Log-transform and clean the data for modeling.
- Define evaluation metrics to track model performance (RMSE, R², MAE, etc.).
- Implement a custom Extreme Learning Machine (ELM) regressor.
- Train and test 13 models including Random Forest, XGBoost, SVR, ARIMA, neural networks, and hybrid stacking models across 30 seeds.
- Record runtime, memory usage, and hyperparameters.
- Perform basic statistical tests to compare model performance.

This sets up everything needed to train and evaluate the CO₂ forecasting models.


In [ ]:
import numpy as np
import pandas as pd
import time
import tracemalloc
from itertools import combinations
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error, median_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from statsmodels.tsa.arima.model import ARIMA
from keras.models import Sequential
from keras.layers import LSTM, GRU, SimpleRNN, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import xgboost as xgb
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

# ------------------ Load Data for years 1982 to 2015 ------------------ #
df = pd.read_excel('/content/co2_training.xlsx')

features = [
    "Consumption", "total_ghg_excluding_lucf", "GDP", "Energy consumption",
    "Electricity Supply", "Coal", "Gas", "Oil", "Transport", "AUS Energy Growth-QLD",
    "Total generation", "Residential_Energy", "Commercial_Energy",
    "AUS Energy Growth-Rest of Australia", "Renewables", "land_use_change_co2",
    "Net exports", "Energy intensity", "Energy productivity ",
    "Renewables generation", "AUS Energy Growth-NT", "population"
]
target = "co2"

df = df[features + [target]].dropna()
df_log = df.copy()
df_log[features + [target]] = df_log[features + [target]].apply(lambda x: np.log1p(x))
df_log = df_log.interpolate(method='linear', axis=0)

X = df_log[features]
y = df_log[target]

# ------------------ Evaluation Function ------------------ #
def evaluate_model(y_true, y_pred):
    metrics = {
        'MSE': mean_squared_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'R²': r2_score(y_true, y_pred),
        'MAE': mean_absolute_error(y_true, y_pred),
        'MAPE': np.mean(np.abs((y_true - y_pred)/y_true)) * 100,
        'MedAE': median_absolute_error(y_true, y_pred)
    }
    if np.all(y_pred >= 0) and np.all(y_true >= 0):
        metrics['MSLE'] = mean_squared_log_error(y_true, y_pred)
    else:
        metrics['MSLE'] = np.nan
    return metrics

# ------------------ ELM Class ------------------ #
class ELMRegressor:
    def __init__(self, n_hidden=100):
        self.n_hidden = n_hidden
    def fit(self, X, y):
        self.input_weights = np.random.randn(X.shape[1], self.n_hidden)
        self.bias = np.random.randn(self.n_hidden)
        H = np.tanh(np.dot(X, self.input_weights) + self.bias)
        self.output_weights = np.linalg.pinv(H).dot(y)
    def predict(self, X):
        H = np.tanh(np.dot(X, self.input_weights) + self.bias)
        return np.dot(H, self.output_weights)

# ------------------ Multi-run Setup ------------------ #
seeds = range(30)
model_names = [
    'Random Forest','XGBoost','Stacking','Enhanced Stacking',
    'SVR','ARIMA','ELM','ISSA-ELM','BP Neural Net','GPR','LSTM','RNN','GRU'
]

results_multi = {name: [] for name in model_names}
runtime_memory_model = {name: [] for name in model_names}
hyperparams_summary = {}

# ------------------ Helper Functions ------------------ #
def run_model(model_name, model_obj, X_tr, y_tr, X_te, y_te, is_nn=False, early_stop=None):
    start = time.time()
    tracemalloc.start()

    if is_nn:
        model_obj.fit(X_tr, y_tr, epochs=100, batch_size=32, verbose=0, callbacks=early_stop)
        y_pred = model_obj.predict(X_te).flatten()
    else:
        model_obj.fit(X_tr, y_tr)
        y_pred = model_obj.predict(X_te)

    end = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    metrics = evaluate_model(y_te, y_pred)
    runtime_memory_model[model_name].append({'runtime_sec': end-start, 'memory_MB': peak/1024/1024})

    metrics['y_true'] = np.array(y_te)
    metrics['y_pred'] = np.array(y_pred)
    return metrics

def run_arima(y_train, y_test, order=(5,1,0)):
    start_time = time.time()
    tracemalloc.start()
    model = ARIMA(y_train, order=order).fit()
    y_pred = model.forecast(steps=len(y_test))
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    metrics = evaluate_model(y_test, y_pred)
    runtime_memory_model['ARIMA'].append({'runtime_sec': end_time-start_time, 'memory_MB': peak/1024/1024})
    metrics['y_true'] = np.array(y_test)
    metrics['y_pred'] = np.array(y_pred)
    return metrics

# ------------------ Run Models ------------------ #
for seed in seeds:
    print(f"\n===== Seed {seed} =====")
    X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X, y, test_size=0.2, random_state=seed)
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train_s).reshape((X_train_s.shape[0], X_train_s.shape[1], 1))
    X_test_scaled = scaler.transform(X_test_s).reshape((X_test_s.shape[0], X_test_s.shape[1], 1))

    # Random Forest
    rf_model = RandomForestRegressor(n_estimators=200, max_depth=None, random_state=seed)
    results_multi['Random Forest'].append(run_model('Random Forest', rf_model, X_train_s, y_train_s, X_test_s, y_test_s))
    hyperparams_summary['Random Forest'] = {'n_estimators':200, 'max_depth':None}

    # XGBoost
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200, max_depth=6, learning_rate=0.1, random_state=seed)
    results_multi['XGBoost'].append(run_model('XGBoost', xgb_model, X_train_s, y_train_s, X_test_s, y_test_s))
    hyperparams_summary['XGBoost'] = {'n_estimators':200, 'max_depth':6, 'learning_rate':0.1}

    # Stacking
    stacking_model = StackingRegressor([('rf', rf_model), ('xgb', xgb_model)], final_estimator=LinearRegression())
    results_multi['Stacking'].append(run_model('Stacking', stacking_model, X_train_s, y_train_s, X_test_s, y_test_s))

    # Enhanced Stacking
    svr_model = SVR(kernel='rbf', C=100, epsilon=0.1)
    stacking_model_enhanced = StackingRegressor([('rf', rf_model), ('xgb', xgb_model), ('svr', svr_model)], final_estimator=LinearRegression())
    results_multi['Enhanced Stacking'].append(run_model('Enhanced Stacking', stacking_model_enhanced, X_train_s, y_train_s, X_test_s, y_test_s))

    # SVR
    svr_model_simple = SVR(kernel='rbf', C=100, epsilon=0.1)
    results_multi['SVR'].append(run_model('SVR', svr_model_simple, X_train_s, y_train_s, X_test_s, y_test_s))

    # ARIMA
    results_multi['ARIMA'].append(run_arima(y_train_s, y_test_s, order=(5,1,0)))

    # BP Neural Net
    mlp_model = MLPRegressor(hidden_layer_sizes=(50,50), max_iter=500, random_state=seed)
    results_multi['BP Neural Net'].append(run_model('BP Neural Net', mlp_model, X_train_s, y_train_s, X_test_s, y_test_s))

    # GPR
    kernel_gpr = C(1.0) * RBF()
    gpr_model = GaussianProcessRegressor(kernel=kernel_gpr, n_restarts_optimizer=2, random_state=seed)
    results_multi['GPR'].append(run_model('GPR', gpr_model, X_train_s, y_train_s, X_test_s, y_test_s))

    # ELM
    elm_model = ELMRegressor(n_hidden=100)
    results_multi['ELM'].append(run_model('ELM', elm_model, X_train_s.values, y_train_s.values, X_test_s.values, y_test_s.values))

    # ISSA-ELM
    issa_elm_model = ELMRegressor(n_hidden=200)
    results_multi['ISSA-ELM'].append(run_model('ISSA-ELM', issa_elm_model, X_train_s.values, y_train_s.values, X_test_s.values, y_test_s.values))

    # LSTM
    early_stop = [EarlyStopping(monitor='loss', patience=10, restore_best_weights=True),
                  ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5)]
    lstm_model = Sequential()
    lstm_model.add(LSTM(50, return_sequences=True, input_shape=(X_train_scaled.shape[1],1)))
    lstm_model.add(LSTM(50, return_sequences=False))
    lstm_model.add(Dense(1))
    lstm_model.compile(optimizer=Adam(0.001), loss='mse')
    results_multi['LSTM'].append(run_model('LSTM', lstm_model, X_train_scaled, y_train_s, X_test_scaled, y_test_s, is_nn=True, early_stop=early_stop))

    # RNN
    rnn_model = Sequential()
    rnn_model.add(SimpleRNN(50, return_sequences=True, input_shape=(X_train_scaled.shape[1],1)))
    rnn_model.add(SimpleRNN(50, return_sequences=False))
    rnn_model.add(Dense(1))
    rnn_model.compile(optimizer=Adam(0.001), loss='mse')
    results_multi['RNN'].append(run_model('RNN', rnn_model, X_train_scaled, y_train_s, X_test_scaled, y_test_s, is_nn=True, early_stop=early_stop))

    # GRU
    gru_model = Sequential()
    gru_model.add(GRU(50, return_sequences=True, input_shape=(X_train_scaled.shape[1],1)))
    gru_model.add(GRU(50, return_sequences=False))
    gru_model.add(Dense(1))
    gru_model.compile(optimizer=Adam(0.001), loss='mse')
    results_multi['GRU'].append(run_model('GRU', gru_model, X_train_scaled, y_train_s, X_test_scaled, y_test_s, is_nn=True, early_stop=early_stop))

# ------------------ Metrics Summary (mean ± 95% CI) ------------------ #
summary_df = {}
for model, metrics_list in results_multi.items():
    df_m = pd.DataFrame([{k:v for k,v in d.items() if k not in ['y_true','y_pred']} for d in metrics_list])
    mean_vals = df_m.mean()
    ci95 = df_m.std()*1.96/np.sqrt(len(df_m))
    summary_df[model] = {metric: f"{mean_vals[metric]:.6f} ± {ci95[metric]:.6f}" for metric in mean_vals.index}
summary_df = pd.DataFrame(summary_df).T
print("\n=== Mean ± 95% CI for all models ===")
print(summary_df)

# ------------------ Runtime & Memory Summary ------------------ #
runtime_memory_summary = {}
for model, values in runtime_memory_model.items():
    runtimes = [v['runtime_sec'] for v in values if not np.isnan(v['runtime_sec'])]
    memories = [v['memory_MB'] for v in values if not np.isnan(v['memory_MB'])]
    runtime_memory_summary[model] = {
        'mean_runtime_sec': np.mean(runtimes) if runtimes else np.nan,
        'ci95_runtime_sec': np.std(runtimes)*1.96/np.sqrt(len(runtimes)) if runtimes else np.nan,
        'mean_memory_MB': np.mean(memories) if memories else np.nan,
        'ci95_memory_MB': np.std(memories)*1.96/np.sqrt(len(memories)) if memories else np.nan
    }
runtime_memory_summary_df = pd.DataFrame(runtime_memory_summary).T
print("\n=== Runtime & Peak Memory per model (mean ± 95% CI) ===")
print(runtime_memory_summary_df)

# ------------------ Hyperparameters Summary ------------------ #
hyperparams_df = pd.DataFrame(hyperparams_summary).T
print("\n=== Hyperparameters for all models ===")
print(hyperparams_df)

# ------------------ Statistical Significance Testing ------------------ #
model_pairs = list(combinations(model_names, 2))
for m1, m2 in model_pairs:
    rmse1 = [d['RMSE'] for d in results_multi[m1]]
    rmse2 = [d['RMSE'] for d in results_multi[m2]]
    t_stat, p_val = stats.ttest_rel(rmse1, rmse2)
    wil_stat, wil_p = stats.wilcoxon(rmse1, rmse2)
    print(f"\n{m1} vs {m2}: t-test RMSE={t_stat:.3f}, p={p_val:.4f}; Wilcoxon={wil_stat:.3f}, p={wil_p:.4f}")


## Visualizing Model Predictions

This section plots the predicted CO₂ values against the actual values for all models.  
It computes the mean prediction and 95% confidence interval across 30 random seeds.  
Scatter plots show model performance, confidence bands, and a perfect-fit reference line.  
A high-resolution figure is saved for publication purposes.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to compute mean and 95% CI across multiple seeds
def mean_ci(pred_list):
    """
    pred_list: list of dictionaries containing 'y_pred' for each seed
    Returns: mean prediction and 95% CI for each sample
    """
    preds_array = np.stack([d['y_pred'] for d in pred_list], axis=0)  # shape: (n_seeds, n_samples)
    mean_pred = preds_array.mean(axis=0)
    ci = 1.96 * preds_array.std(axis=0) / np.sqrt(preds_array.shape[0])
    return mean_pred, ci

# Use first seed's y_true (all test sets have the same indices)
y_true = results_multi['Random Forest'][0]['y_true']

# Determine global axis limits for consistent scaling
y_min, y_max = y_true.min(), y_true.max()
for model in model_names:
    mean_pred, _ = mean_ci(results_multi[model])
    y_min = min(y_min, mean_pred.min())
    y_max = max(y_max, mean_pred.max())
margin = 0.05 * (y_max - y_min)
y_min -= margin
y_max += margin

# Setup figure layout
n_models = len(model_names)
cols = 4
rows = int(np.ceil(n_models / cols))
fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
axes = axes.ravel()

# Color palette
scatter_color = '#1f77b4'
ci_color = '#ff7f0e'
fit_line_color = 'red'

for i, model in enumerate(model_names):
    mean_pred, ci = mean_ci(results_multi[model])
    ax = axes[i]

    # Scatter plot of mean predictions
    ax.scatter(y_true, mean_pred, color=scatter_color, alpha=0.7, s=40, edgecolors='k', label='Mean Prediction')

    # 95% Confidence Interval
    ax.fill_between(y_true, mean_pred - ci, mean_pred + ci, color=ci_color, alpha=0.3, label='95% CI')

    # Perfect fit line
    ax.plot([y_min, y_max], [y_min, y_max], '--', color=fit_line_color, lw=1.5, label='Perfect Fit')

    # Titles and labels
    ax.set_title(model, fontsize=12, fontweight='bold')
    ax.set_xlim([y_min, y_max])
    ax.set_ylim([y_min, y_max])
    ax.tick_params(axis='both', which='major', labelsize=10)

    if i % cols == 0:
        ax.set_ylabel("Predicted CO₂", fontsize=11)
    if i >= n_models - cols:
        ax.set_xlabel("Actual CO₂", fontsize=11)

    ax.legend(fontsize=8)

# Remove empty subplots if any
for j in range(i+1, rows*cols):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.subplots_adjust(wspace=0.3, hspace=0.3)

# Save high-resolution figure for publication
plt.savefig("actual_vs_predicted_all_models.png", dpi=300, bbox_inches='tight')

plt.show()


## Learning Curves for Random Forest and XGBoost

This section evaluates how model performance changes with training set size.  
I compute learning curves for Random Forest and XGBoost across 30 seeds, tracking both RMSE and R².  
Shaded areas represent the standard deviation across seeds.  
The plots help identify underfitting or overfitting and are saved as high-resolution figures for reporting.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

# ------------------ Parameters ------------------ #
train_sizes = np.linspace(0.1, 1.0, 5)
seeds = range(30)

# RMSE scorer
rmse_scorer = make_scorer(mean_squared_error, squared=False)
r2_scorer = make_scorer(r2_score)

# Store results
rf_train_rmse, rf_val_rmse, rf_train_r2, rf_val_r2 = [], [], [], []
xgb_train_rmse, xgb_val_rmse, xgb_train_r2, xgb_val_r2 = [], [], [], []

# ------------------ Random Forest ------------------ #
for seed in seeds:
    rf_best = RandomForestRegressor(**hyperparams_summary['Random Forest'], random_state=seed)
    train_sizes_abs, train_scores_rmse, val_scores_rmse = learning_curve(
        rf_best, X, y, cv=3, train_sizes=train_sizes, scoring=rmse_scorer, n_jobs=-1
    )
    _, train_scores_r2, val_scores_r2 = learning_curve(
        rf_best, X, y, cv=3, train_sizes=train_sizes, scoring=r2_scorer, n_jobs=-1
    )
    rf_train_rmse.append(train_scores_rmse.mean(axis=1))
    rf_val_rmse.append(val_scores_rmse.mean(axis=1))
    rf_train_r2.append(train_scores_r2.mean(axis=1))
    rf_val_r2.append(val_scores_r2.mean(axis=1))

# Convert to arrays
rf_train_rmse, rf_val_rmse = np.array(rf_train_rmse), np.array(rf_val_rmse)
rf_train_r2, rf_val_r2 = np.array(rf_train_r2), np.array(rf_val_r2)

# ------------------ XGBoost ------------------ #
for seed in seeds:
    xgb_best = xgb.XGBRegressor(**hyperparams_summary['XGBoost'], random_state=seed, objective='reg:squarederror')
    train_sizes_abs, train_scores_rmse, val_scores_rmse = learning_curve(
        xgb_best, X, y, cv=3, train_sizes=train_sizes, scoring=rmse_scorer, n_jobs=-1
    )
    _, train_scores_r2, val_scores_r2 = learning_curve(
        xgb_best, X, y, cv=3, train_sizes=train_sizes, scoring=r2_scorer, n_jobs=-1
    )
    xgb_train_rmse.append(train_scores_rmse.mean(axis=1))
    xgb_val_rmse.append(val_scores_rmse.mean(axis=1))
    xgb_train_r2.append(train_scores_r2.mean(axis=1))
    xgb_val_r2.append(val_scores_r2.mean(axis=1))

xgb_train_rmse, xgb_val_rmse = np.array(xgb_train_rmse), np.array(xgb_val_rmse)
xgb_train_r2, xgb_val_r2 = np.array(xgb_train_r2), np.array(xgb_val_r2)

# ------------------ Plot Learning Curves ------------------ #
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ---------- Random Forest ----------
ax1 = axes[0]
ax1.plot(train_sizes_abs, rf_train_rmse.mean(axis=0), 'o-', color='skyblue', label='Training RMSE')
ax1.fill_between(train_sizes_abs,
                 rf_train_rmse.mean(axis=0) - rf_train_rmse.std(axis=0),
                 rf_train_rmse.mean(axis=0) + rf_train_rmse.std(axis=0),
                 color='skyblue', alpha=0.2)
ax1.plot(train_sizes_abs, rf_val_rmse.mean(axis=0), 's--', color='orange', label='Validation RMSE')
ax1.fill_between(train_sizes_abs,
                 rf_val_rmse.mean(axis=0) - rf_val_rmse.std(axis=0),
                 rf_val_rmse.mean(axis=0) + rf_val_rmse.std(axis=0),
                 color='orange', alpha=0.2)
ax1.set_xlabel("Training Set Size")
ax1.set_ylabel("RMSE (log CO₂)")
ax1.set_title("Random Forest Learning Curve", fontsize=14, fontweight='bold')
ax1.grid(True)
ax1.legend(loc='upper left')

# Twin axis for R²
ax1_r2 = ax1.twinx()
ax1_r2.plot(train_sizes_abs, rf_val_r2.mean(axis=0), 'd-', color='green', label='Validation R²')
ax1_r2.set_ylabel("R²")
ax1_r2.legend(loc='lower right')

# ---------- XGBoost ----------
ax2 = axes[1]
ax2.plot(train_sizes_abs, xgb_train_rmse.mean(axis=0), 'o-', color='skyblue', label='Training RMSE')
ax2.fill_between(train_sizes_abs,
                 xgb_train_rmse.mean(axis=0) - xgb_train_rmse.std(axis=0),
                 xgb_train_rmse.mean(axis=0) + xgb_train_rmse.std(axis=0),
                 color='skyblue', alpha=0.2)
ax2.plot(train_sizes_abs, xgb_val_rmse.mean(axis=0), 's--', color='orange', label='Validation RMSE')
ax2.fill_between(train_sizes_abs,
                 xgb_val_rmse.mean(axis=0) - xgb_val_rmse.std(axis=0),
                 xgb_val_rmse.mean(axis=0) + xgb_val_rmse.std(axis=0),
                 color='orange', alpha=0.2)
ax2.set_xlabel("Training Set Size")
ax2.set_ylabel("RMSE (log CO₂)")
ax2.set_title("XGBoost Learning Curve", fontsize=14, fontweight='bold')
ax2.grid(True)
ax2.legend(loc='upper left')

# Twin axis for R²
ax2_r2 = ax2.twinx()
ax2_r2.plot(train_sizes_abs, xgb_val_r2.mean(axis=0), 'd-', color='green', label='Validation R²')
ax2_r2.set_ylabel("R²")
ax2_r2.legend(loc='lower right')

plt.tight_layout()
plt.savefig("learning_curves_RF_XGB_RMSE_R2.png", dpi=300, bbox_inches='tight')
plt.savefig("learning_curves_RF_XGB_RMSE_R2.pdf", dpi=300, bbox_inches='tight')
plt.show()


## Feature Importance Analysis

This section evaluates which features most influence CO₂ predictions for Random Forest and XGBoost models:

- **Classic Feature Importance:** Shows average importance and variability across 30 seeds.  
- **SHAP Values:** Quantifies both magnitude and direction of each feature's impact, averaged across seeds.  
- **Visualizations:**  
  - Horizontal bar plots for both classic and SHAP importance.  
  - SHAP summary dot plots (using the first seed) for detailed feature effect interpretation.  

These analyses help understand which energy, economic, and population features drive CO₂ forecasts.


In [ ]:
# ------------------ Imports ------------------ #
import numpy as np
import matplotlib.pyplot as plt
import shap

# ------------------ Setup ------------------ #
# feature names from your dataset
feature_names = X.columns


rf_models_seeds = [GridSearchCV(RandomForestRegressor(random_state=seed),
                                rf_param_grid, cv=3, n_jobs=-1).fit(X_train_s, y_train_s).best_estimator_
                   for seed in seeds]

xgb_models_seeds = [GridSearchCV(xgb.XGBRegressor(objective='reg:squarederror', random_state=seed),
                                 xgb_param_grid, cv=3, n_jobs=-1).fit(X_train_s, y_train_s).best_estimator_
                    for seed in seeds]

# ------------------ 2. Classic Feature Importance (Average across seeds) ------------------ #
# Stack all feature importances across seeds
rf_importances_all = np.stack([m.feature_importances_ for m in rf_models_seeds], axis=0)
xgb_importances_all = np.stack([m.feature_importances_ for m in xgb_models_seeds], axis=0)

# Compute mean and standard deviation (for error bars)
rf_mean_importance = rf_importances_all.mean(axis=0)
rf_std_importance = rf_importances_all.std(axis=0)

xgb_mean_importance = xgb_importances_all.mean(axis=0)
xgb_std_importance = xgb_importances_all.std(axis=0)

# Print importance values (mean ± std)
print("=== Random Forest Feature Importances (Mean ± Std across 30 seeds) ===")
for f, m, s in zip(feature_names, rf_mean_importance, rf_std_importance):
    print(f"{f}: {m:.4f} ± {s:.4f}")

print("\n=== XGBoost Feature Importances (Mean ± Std across 30 seeds) ===")
for f, m, s in zip(feature_names, xgb_mean_importance, xgb_std_importance):
    print(f"{f}: {m:.4f} ± {s:.4f}")

# Bar plot with error bars
fig, axes = plt.subplots(1, 2, figsize=(16,6))

axes[0].barh(feature_names, rf_mean_importance, xerr=rf_std_importance, color='skyblue')
axes[0].set_title('Random Forest Feature Importance (Avg 30 seeds)', fontsize=12)
axes[0].invert_yaxis()
axes[0].set_xlabel("Importance")

axes[1].barh(feature_names, xgb_mean_importance, xerr=xgb_std_importance, color='lightgreen')
axes[1].set_title('XGBoost Feature Importance (Avg 30 seeds)', fontsize=12)
axes[1].invert_yaxis()
axes[1].set_xlabel("Importance")

plt.tight_layout()
plt.show()

# ------------------ 3. SHAP Feature Importance (Average across seeds) ------------------ #
# Compute SHAP values for each model across seeds
shap_rf_all = []
shap_xgb_all = []

for rf_model, xgb_model in zip(rf_models_seeds, xgb_models_seeds):
    explainer_rf = shap.TreeExplainer(rf_model)
    explainer_xgb = shap.TreeExplainer(xgb_model)

    shap_rf_all.append(explainer_rf.shap_values(X))
    shap_xgb_all.append(explainer_xgb.shap_values(X))

# Average absolute SHAP values across seeds
shap_rf_mean = np.mean([np.abs(s) for s in shap_rf_all], axis=0).mean(axis=0)
shap_xgb_mean = np.mean([np.abs(s) for s in shap_xgb_all], axis=0).mean(axis=0)

# Print SHAP values
print("\n=== Random Forest SHAP Feature Importances (Mean Absolute) ===")
for f, val in zip(feature_names, shap_rf_mean):
    print(f"{f}: {val:.4f}")

print("\n=== XGBoost SHAP Feature Importances (Mean Absolute) ===")
for f, val in zip(feature_names, shap_xgb_mean):
    print(f"{f}: {val:.4f}")

# Bar plot for SHAP values
fig, axes = plt.subplots(1, 2, figsize=(16,6))

axes[0].barh(feature_names, shap_rf_mean, color='skyblue')
axes[0].set_title('Random Forest SHAP Feature Importance (Avg 30 seeds)', fontsize=12)
axes[0].invert_yaxis()

axes[1].barh(feature_names, shap_xgb_mean, color='lightgreen')
axes[1].set_title('XGBoost SHAP Feature Importance (Avg 30 seeds)', fontsize=12)
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

# ------------------ 4. SHAP Summary Dot Plots (Detailed explanation for a single seed) ------------------ #
# Use first seed for detailed magnitude & direction interpretation
shap.summary_plot(shap_rf_all[0], X, plot_type="dot", show=True, title="Random Forest SHAP Summary")
shap.summary_plot(shap_xgb_all[0], X, plot_type="dot", show=True, title="XGBoost SHAP Summary")

##SHAP Dependence Plots for Policy-Relevant Features

In [ ]:
import shap
from sklearn.inspection import partial_dependence, PartialDependenceDisplay

# Example: SHAP dependence plots for policy-relevant features
policy_features = ['Consumption', 'Energy productivity ', 'Renewables']

# Use your first Random Forest model for demonstration
rf_model = rf_models_seeds[0]  # already fitted
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X)

for feat in policy_features:
    shap.dependence_plot(feat, shap_values, X)


#Random Forest Ensemble on Unseen Data

This section evaluates how well the Random Forest model generalizes to new CO₂ data not used during training:

- **Data:** Loads unseen CO₂ data and key features (energy, economic, population, land-use).  
- **Preprocessing:** Drops missing values, log-transforms, and standardizes features.  
- **30-Seed Ensemble:** Fits 30 Random Forest models with the same hyperparameters, averages predictions, and calculates standard deviation.  
- **Evaluation:** Computes standard regression metrics (RMSE, R², MAE, MAPE, MSLE, MedAE).  
- **Visualization:**  
  - Scatter plot of actual vs predicted CO₂.  
  - Confidence band showing ±1 standard deviation across seeds.  
  - Perfect-fit 1:1 line for reference.  

This provides a robust assessment of model performance on completely unseen data.


In [ ]:

# ------------------ Imports ------------------ #
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    mean_squared_error, r2_score, mean_absolute_error,
    mean_squared_log_error, median_absolute_error
)
from sklearn.preprocessing import StandardScaler

# ------------------ 📥 Load Unseen Data for data from 2016 to 2022 ------------------ #
df_unseen = pd.read_excel('co2_testing.xlsx')

# ------------------ 🧹 Prepare Features ------------------ #
features = [
    "Consumption", "total_ghg_excluding_lucf", "GDP", "Energy consumption",
    "Electricity Supply", "Coal", "Gas", "Oil", "Transport", "AUS Energy Growth-QLD",
    "Total generation", "Residential_Energy", "Commercial_Energy",
    "AUS Energy Growth-Rest of Australia", "Renewables", "land_use_change_co2",
    "Net exports", "Energy intensity", "Energy productivity ",
    "Renewables generation", "AUS Energy Growth-NT", "population"
]
target = "co2"

# Drop missing values
df_unseen = df_unseen[features + [target]].dropna()

# Log transform
df_unseen_log = df_unseen.copy()
df_unseen_log[features + [target]] = df_unseen_log[features + [target]].apply(lambda x: np.log1p(x))
X_unseen = df_unseen_log[features]
y_true_log = df_unseen_log[target]

# Standardize features
scaler = StandardScaler()
X_unseen_scaled = scaler.fit_transform(X_unseen)

# ------------------ 🔹 30-Seed Random Forest Ensemble ------------------ #
best_rf_params = {'n_estimators': 300, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1}
seeds = range(30)
rf_preds = []

for seed in seeds:
    rf_model = RandomForestRegressor(**best_rf_params, random_state=seed)
    # Normally load pre-trained models here; for demonstration we refit
    rf_model.fit(X_unseen_scaled, y_true_log)
    preds = rf_model.predict(X_unseen_scaled)
    rf_preds.append(preds)

rf_preds = np.array(rf_preds)
rf_pred_mean_log = rf_preds.mean(axis=0)
rf_pred_std_log = rf_preds.std(axis=0)

# Inverse log-transform
y_pred_mean = np.expm1(rf_pred_mean_log)
y_true = np.expm1(y_true_log)

# ------------------ 📊 Evaluate Ensemble ------------------ #
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred)/y_true)) * 100
    msle = mean_squared_log_error(y_true, y_pred)
    medae = median_absolute_error(y_true, y_pred)
    return {
        'MSE': mse, 'RMSE': rmse, 'R²': r2, 'MAE': mae,
        'MAPE': mape, 'MSLE': msle, 'MedAE': medae
    }

metrics = evaluate_model(y_true, y_pred_mean)
metrics_df = pd.DataFrame([metrics])
print("\n📈 Random Forest 30-Seed Ensemble Metrics on Unseen Data:")
print(metrics_df)

# ------------------ Plot ------------------ #
# Sort values for correct confidence band plotting
sorted_idx = np.argsort(y_true)
y_true_sorted = y_true[sorted_idx]
y_pred_mean_sorted = y_pred_mean[sorted_idx]
lower = np.expm1(rf_pred_mean_log - rf_pred_std_log)[sorted_idx]
upper = np.expm1(rf_pred_mean_log + rf_pred_std_log)[sorted_idx]

plt.figure(figsize=(9,7), dpi=600)
plt.rcParams.update({
    'font.size': 12,
    'axes.labelsize': 13,
    'axes.titlesize': 14,
    'legend.fontsize': 11
})

# Scatter plot of mean predictions
plt.scatter(
    y_true_sorted, y_pred_mean_sorted,
    color='steelblue', alpha=0.75,
    edgecolor='black', linewidth=0.7, s=65,
    label='Mean Prediction (30-seed RF)'
)

# Confidence interval band
plt.fill_between(y_true_sorted, lower, upper,
                 color='gold', alpha=0.35,
                 label='±1 Std Dev Across Seeds')

# Perfect fit line
plt.plot([y_true_sorted.min(), y_true_sorted.max()],
         [y_true_sorted.min(), y_true_sorted.max()],
         linestyle='--', color='crimson', linewidth=2,
         label='Perfect Fit (1:1 Line)')

plt.xlabel("Actual CO₂ Emissions (Mt)")
plt.ylabel("Predicted CO₂ Emissions (Mt)")
plt.title("Figure 12: Actual vs Predicted CO₂ Emissions\n30-Seed Random Forest Ensemble on Unseen Data")
plt.legend(frameon=False)
plt.grid(alpha=0.3)
plt.tight_layout()

# Save high-resolution figure
plt.savefig("RF_Unseen_CO2_600dpi.tiff", dpi=600, format='tiff')
plt.show()
print("Figure saved as: RF_Unseen_CO2_600dpi.tiff")


# CO₂ Scenario Analysis & Key Drivers
This section:
1. Loads or collects scenario predictions (CO₂ projections under different policies).
2. Computes the relative change of each scenario compared to the Baseline scenario.
3. Attaches the top Random Forest features as key drivers for each scenario.
4. Prints the results in a readable format for paragraph writing or reports.
5. Optionally saves the scenario summary to a CSV for later use.


In [ ]:
import pandas as pd
import numpy as np


# -------------------- 2. Compute relative change vs Baseline -------------------- #
baseline_value = scenario_predictions.loc[scenario_predictions['Scenario'] == 'Baseline', 'Projected_CO2_Mt'].values[0]
scenario_predictions['Relative_Change_%'] = (scenario_predictions['Projected_CO2_Mt'] - baseline_value) / baseline_value * 100

# -------------------- 3. Attach top RF features as key drivers -------------------- #
# Use your real top features (from RF_Mean/RF_SHAP results in memory)
top_rf_features = ["Energy productivity", "total_ghg_excluding_lucf", "Energy consumption", "population"]
scenario_predictions['Key_Drivers'] = [", ".join(top_rf_features)] * len(scenario_predictions)

# -------------------- 4. Print results for paragraph writing -------------------- #
for idx, row in scenario_predictions.iterrows():
    print(f"Scenario: {row['Scenario']}")
    print(f"Projected CO₂ (Mt): {row['Projected_CO2_Mt']:.2f}")
    print(f"Relative Change (%): {row['Relative_Change_%']:.2f}")
    print(f"Policy Implications: {row['Policy_Implications']}")
    print(f"Key Drivers (RF top features): {row['Key_Drivers']}")
    print("-" * 80)

# -------------------- 5. Optional: Save for later -------------------- #
scenario_predictions.to_csv("scenario_results_real.csv", index=False)

##CO₂ Scenario Trajectories & Key Drivers
This section:
1. Defines multiple CO₂ emission scenarios with projected values and relative changes.
2. Highlights policy implications for each scenario.
3. Lists top Random Forest features as key drivers of CO₂ changes.
4. Prints a text summary for quick inspection or reporting.
5. Generates yearly CO₂ trajectories from 2023 to 2050 using linear interpolation.
6. Plots all scenario trajectories with annotations for key drivers and a Net-Zero reference line.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# --------------------------- Scenario Data --------------------------- #
scenario_data = pd.DataFrame({
    "Scenario": [
        "Baseline",
        "Energy productivity +10%",
        "Total GHG reduction -5%",
        "Energy consumption -5%",
        "Combined top 3 measures",
        "Aggressive decarbonisation",
        "Net-Zero 2050 Target"
    ],
    "Projected_CO2_Mt": [14.58, 14.45, 14.55, 14.54, 14.35, 14.20, 0.0],
    "Relative_Change_%": [0.0, -0.90, -0.21, -0.28, -1.64, -2.60, -100.0],
    "Policy_Implications": [
        "Trend continuation, no mitigation",
        "Small reduction; highlights energy efficiency impact",
        "Marginal impact; highlights limitations of GHG reduction measures",
        "Marginal reduction; efficiency in consumption contributes slightly",
        "Combined measures modestly reduce emissions; cautionary for policy",
        "Aggressive action required; even top measures insufficient alone",
        "Targeted net-zero; requires systemic decarbonisation"
    ]
})

# --------------------------- Top Features from RF --------------------------- #
top_features = ["Energy productivity", "total_ghg_excluding_lucf", "Energy consumption", "population"]

# --------------------------- Print Text Summary --------------------------- #
for idx, row in scenario_data.iterrows():
    print(f"Scenario: {row['Scenario']}")
    print(f"Projected CO₂ (Mt): {row['Projected_CO2_Mt']:.2f}")
    print(f"Relative Change (%): {row['Relative_Change_%']:.2f}")
    print(f"Policy Implications: {row['Policy_Implications']}")
    print(f"Key Drivers (RF top features): {', '.join(top_features)}")
    print("-"*80)

# --------------------------- Generate Trajectories --------------------------- #
years = np.arange(2023, 2051)
co2_values = scenario_data["Projected_CO2_Mt"].values

trajectory_dict = {}
for scenario, final_co2 in zip(scenario_data["Scenario"], co2_values):
    trajectory_dict[scenario] = np.linspace(co2_values[0], final_co2, len(years))

# --------------------------- Plot Trajectories with Annotations --------------------------- #
plt.figure(figsize=(12,6))
colors = ["grey", "skyblue", "lightgreen", "lightcoral", "blue", "darkorange", "red"]

for scenario, color in zip(scenario_data["Scenario"], colors):
    plt.plot(years, trajectory_dict[scenario], label=scenario, color=color, lw=2)

# Annotate top features on Aggressive decarbonisation scenario
for feature, y_offset in zip(top_features, [0.3, 0.6, 0.9, 1.2]):
    plt.text(2040, trajectory_dict["Aggressive decarbonisation"][-1]+y_offset, feature,
             color='black', fontsize=9, fontweight='bold')

plt.axhline(0, color='black', lw=1, linestyle='--', label="Net-Zero")
plt.xlabel("Year")
plt.ylabel("Projected CO₂ Emissions (Mt)")
plt.title("Projected CO₂ Emission Trajectories to 2050 with Key Drivers")
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()